In [1]:
project_name="cnnclf"

In [2]:
import os

In [3]:
%pwd

'f:\\ML_Ops_learning\\Kidney_Disease_Classification_MLflow\\notebooks'

In [4]:
os.chdir("../")

In [5]:
%pwd

'f:\\ML_Ops_learning\\Kidney_Disease_Classification_MLflow'

## config/config.yaml

In [6]:
%%writefile -a config/config.yaml

training:
  root_dir: artifacts/training
  trained_model_path: artifacts/training/model.h5

Appending to config/config.yaml


# Entity Folder
1. Goto entity folder  src-> wine_quality -> entity-> __init__.py 
2. Paste below code in __init__.py constructor file

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

into file

In [8]:
project_name

'cnnclf'

In [9]:
%%writefile -a src/{project_name}/entity/__init__.py

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

Appending to src/cnnclf/entity/__init__.py


# Config folder 
1. Copy get_model_trainer_config function 
2. Goto config folder src->wine_quality->config->configuration.py paste under ConfigurationManager class
3. Import ModelTrainerConfig from textSummarizer.entity using this line  
from wine_quality.entity import ModelTrainerConfig

In [8]:
from cnnclf.constants import *
from cnnclf.utils import read_yaml, create_directories

In [9]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            ):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "kidney-ct-scan-image")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config
        

In [10]:
project_name

'cnnclf'

In [11]:
%%writefile -a src/{project_name}/config/__init__.py

from cnnclf.entity import TrainingConfig
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "kidney-ct-scan-image")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config



Appending to src/cnnclf/config/__init__.py


# Component folder
1. Copy entire ModelTrainer class
2. Goto Component folder [ src-> textSummarizer -> components - > model_trainer.py ]
2. Paste ModelTrainer class in model_trainer.py
3. Import ModelTrainerConfig from entity using this line 

from textSummarizer.entity import ModelTrainerConfig

In [12]:
project_name

'cnnclf'

In [14]:
#%%writefile -a src/{project_name}/components/model_training.py
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [16]:
# %%writefile -a src/{project_name}/components/model_training.py
# from cnnclf.entity import TrainingConfig

In [18]:
#%%writefile -a src/{project_name}/components/model_training.py

class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)



    
    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

Appending to src/cnnclf/components/model_training.py


# Pipeline folder

1. Goto Component folder [ src-> textSummarizer -> pipeline - > stage_04_model_trainer.py ]

2. Paste below code in stage_04_model_trainer.py

In [12]:
# ''' 
# Example
# from wine_quality import logger
# from wine_quality.config import ConfigurationManager
# from wine_quality.components.model_trainer import ModelTrainer
# class ModelTrainerTrainingPipeline:
#     def __init__(self):
#         pass
#     def main(self):
        
# '''

3. Copy try block code 
4. Paste in main function in stage_04_model_trainer.py

5. Import ModelTrainerConfig from config.configuration using this line  
from wine_quality.config.configuration import ConfigurationManager

6. Import ModelTrainer  from  components.model_trainer  using this line  
from wine_quality.components.model_trainer  import ModelTrainer 

In [20]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2023-10-28 09:39:32,195] - cnnclf - INFO - __init__ file - LineNum:31   - yaml file: config\config.yaml loaded successfully
[2023-10-28 09:39:32,202] - cnnclf - INFO - __init__ file - LineNum:31   - yaml file: params.yaml loaded successfully
[2023-10-28 09:39:32,205] - cnnclf - INFO - __init__ file - LineNum:50   - created directory at: artifacts
[2023-10-28 09:39:32,208] - cnnclf - INFO - __init__ file - LineNum:50   - created directory at: artifacts\training
Found 93 images belonging to 2 classes.
Found 372 images belonging to 2 classes.
23/23 [==============================] - 156s 7s/step - loss: 13.2436 - accuracy: 0.5562 - val_loss: 10.9599 - val_accuracy: 0.4375


c:\Users\win10\.conda\envs\cnncls\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [14]:
# '''
# Example
# from wine_quality.config import ConfigurationManager
# from wine_quality.conponents.model_trainer import ModelTrainer
# from wine_quality import logger


# class ModelTrainerTrainingPipeline:
#     def __init__(self):
#         pass

#     def main(self):
#         config = ConfigurationManager()
#         model_trainer_config = config.get_model_trainer_config()
#         model_trainer_config = ModelTrainer(config=model_trainer_config)
#         model_trainer_config.train()
# '''

into file

In [21]:
project_name

'cnnclf'

In [22]:
%%writefile -a src/{project_name}/pipeline/stage_03_model_training.py
from cnnclf import logger
from cnnclf.config import ConfigurationManager
from cnnclf.components.model_training import Training

class ModelTrainingPipeline:
    def __init__(self):
        pass

    def main(self):
        config = ConfigurationManager()
        training_config = config.get_training_config()
        training = Training(config=training_config)
        training.get_base_model()
        training.train_valid_generator()
        training.train()

Appending to src/cnnclf/pipeline/stage_03_model_training.py


main file

In [15]:
# '''
# 1. Goto main.py and paste below code
# STAGE_NAME = "Model Trainer stage"
# try: 
#    logger.info(f"*******************")
#    logger.info(f">>>>>> stage {STAGE_NAME} started <<<<<<")
#    model_trainer = ModelTrainerTrainingPipeline()
#    model_trainer.main()
#    logger.info(f">>>>>> stage {STAGE_NAME} completed <<<<<<\n\nx==========x")
# except Exception as e:
#         logger.exception(e)
#         raise e

# 2. Run main.py 
# '''

In [23]:
%%writefile -a src/{project_name}/pipeline/stage_03_model_training.py
STAGE_NAME = "Training"
try: 
   logger.info(f"*******************")
   logger.info(f">>>>>> stage {STAGE_NAME} started <<<<<<")
   model_trainer = ModelTrainingPipeline()
   model_trainer.main()
   logger.info(f">>>>>> stage {STAGE_NAME} completed <<<<<<\n\nx==========x")
except Exception as e:
        logger.exception(e)
        raise e

Appending to src/cnnclf/pipeline/stage_03_model_training.py


In [ ]:
%%writefile -a main.py

STAGE_NAME = "Training"
try: 
   logger.info(f"*******************")
   logger.info(f">>>>>> stage {STAGE_NAME} started <<<<<<")
   model_trainer = ModelTrainingPipeline()
   model_trainer.main()
   logger.info(f">>>>>> stage {STAGE_NAME} completed <<<<<<\n\nx==========x")
except Exception as e:
        logger.exception(e)
        raise e